LangGraph is a framework for modelling agent workflows as graphs, using **nodes** (Python functions that perform computations), **edges** (Python functions that determine the next node to execute), and a **state** (a shared data structure representing the application's current snapshot).

Here's a breakdown of key LangGraph concepts:

**Core Components:**

*   **State**: A shared data structure, typically a `TypedDict` or Pydantic `BaseModel`, that holds the current state of the application and is passed between nodes.
    *   The state's schema is the input schema to all nodes and edges in the graph.
    *   The state is updated by reducer functions associated with each key in the state. If no reducer is specified, updates to a key override previous values.
    *   The `Annotated` type can specify reducer functions for state keys.
    *   State can also have multiple schemas that include private state channels for internal node communication, and input/output schemas that are subsets of an overall schema.
    *   Nodes can write to any state channel defined in the graph state, even if not included in the input schema, and can declare additional state channels if the schema is defined.
*   **Nodes**: Python functions that contain the logic of the agents and receive the current state as input, perform actions, and return an updated state.
    *   Nodes can be synchronous or asynchronous.
    *   The first positional argument of a node is the state, and the second (optional) argument is a "config".
    *   Nodes can be added with or without a specified name. If no name is given, it defaults to the function name.
    *   The `START` node represents the user input to the graph, and the `END` node represents a terminal node.
*   **Edges**: Python functions that determine which node to execute next based on the current state.
    *   Edges can be conditional branches or fixed transitions.
    *   A node can have multiple outgoing edges, in which case all destination nodes will be executed in parallel in the next superstep.
    *   Normal edges go directly from one node to the next, while conditional edges use a function to determine the next node.
    *   The `START` node is used to define the entry point to a graph, and conditional entry points can be used to start at different nodes based on logic.

**Graph Execution:**

*   LangGraph uses message passing, where nodes send messages along edges to other nodes, which then execute their functions. This process occurs in discrete "super-steps".
*   A super-step is an iteration over the graph nodes. Nodes that run in parallel are part of the same super-step, while nodes that run sequentially are in separate super-steps.
*   Nodes begin in an inactive state and become active when they receive a new message on an incoming edge. Active nodes run their function and respond with updates.
*   The graph terminates when all nodes are inactive and no messages are in transit.

**State Management and Updates:**

*   **Reducers** determine how updates from nodes are applied to the state. Each key in the state has its reducer function.
    *   The default reducer overrides the existing value with the update.
    *   The `Annotated` type is used to specify custom reducer functions, such as `operator.add` to append to a list.
    *   The `add_messages` reducer is used to handle lists of messages, keeping track of message IDs and overwriting existing messages if updated.
    *   Messages are deserialized into LangChain `Message` objects using `add_messages`.
*  `MessagesState` is a prebuilt state that includes a list of `AnyMessage` objects and uses the `add_messages` reducer.

**Additional Features:**

*   **Compilation:** Graphs must be compiled before they can be used. Compilation involves basic checks on the graph structure and allows specification of runtime arguments.
*   **Send**: The `Send` object allows for dynamic creation of edges and different state versions, which is useful in map-reduce patterns.
*   **Command**: The `Command` object can be returned from a node function to combine state updates and control flow.
    *   `Command` should be used when state updates and routing to another node are required in the same node. Use conditional edges for conditional routing without state updates.
    *   When using `Command`, type annotations with the list of node names for routing are necessary.
    *   `Command` is also used to update graph state from within tools.
*   **Persistence:** LangGraph provides built-in persistence for an agent's state using checkpointers which saves snapshots of the graph state at each superstep, allowing the graph to be resumed at any time.
*   **Threads:** Threads represent individual sessions or conversations with the graph.
*   **Storage:** LangGraph provides built-in document storage using the `BaseStore` interface, which enables cross-thread persistence for features such as knowledge bases.
*   **Graph Migrations:** LangGraph handles migrations of graph definitions, supporting most topology changes and state modifications while using a checkpointer.
*   **Configuration:** Parts of a graph can be marked as configurable to enable switching between models or system prompts. Configuration is passed via the `configurable` key in the config dictionary and accessed in nodes.
    *   A recursion limit sets the maximum number of super-steps the graph can execute, which can be set via the `recursion_limit` config key.
*   **Interrupt:** The `interrupt` function pauses the graph at specific points to collect user input and allow manual decisions before resuming.  Resuming the graph is done with the `Command` object by setting the `resume` key to the value returned by interrupt.
*   **Breakpoints**: Breakpoints pause graph execution and enable step-by-step debugging using the persistence layer. Breakpoints can also be used in human-in-the-loop workflows.
*   **Subgraphs:** Subgraphs are graphs used as nodes in other graphs, enabling encapsulation and reuse of nodes.
    *   Subgraphs can be added as a compiled subgraph or a function that invokes the subgraph.
    *   If the parent graph and subgraph share state keys, you can use the compiled subgraph directly. Otherwise, invoke the subgraph using a function.
*   **Visualization**: LangGraph includes built-in ways to visualize graphs.
*  **Streaming**: LangGraph supports streaming updates from graph nodes, including streaming tokens from LLM calls.


In [8]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

class InputState(TypedDict):
    user_input: str

class OutputState(TypedDict):
    graph_output: str

class OverallState(TypedDict):
    foo: str
    user_input: str
    graph_output: str

class PrivateState(TypedDict):
    bar: str

def node_1(state: InputState) -> OverallState:
    # Write to OverallState
    return {"foo": state["user_input"] + " name"}

def node_2(state: OverallState) -> PrivateState:
    # Read from OverallState, write to PrivateState
    return {"bar": state["foo"] + " is"}

def node_3(state: PrivateState) -> OutputState:
    # Read from PrivateState, write to OutputState
    return {"graph_output": state["bar"] + " Lance"}

builder = StateGraph(OverallState,input=InputState,output=OutputState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_edge("node_3", END)

graph = builder.compile()
graph.invoke({"user_input":"My"})
{'graph_output': 'My name is Lance'}

{'graph_output': 'My name is Lance'}

In [6]:
from IPython.display import Image

print(graph.get_graph().draw_ascii())

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
  +--------+   
  | node_1 |   
  +--------+   
      *        
      *        
      *        
  +--------+   
  | node_2 |   
  +--------+   
      *        
      *        
      *        
  +--------+   
  | node_3 |   
  +--------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [9]:
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

class State(TypedDict):
    foo: int
    bar: Annotated[list[str], add]

In [26]:
from langgraph.graph import MessagesState

class State(MessagesState):
    documents: list[str]

In [30]:
x = State(documents=["a","b"], messages=["c","d"])

In [31]:
x

{'documents': ['a', 'b'], 'messages': ['c', 'd']}

In [1]:
from langgraph.types import Send


In [4]:
from typing import TypedDict
from typing import Annotated
import operator
from langgraph.types import Send
from langgraph.graph import END, START
from langgraph.graph import StateGraph

class OverallState(TypedDict):
    subjects: list[str]
    jokes: Annotated[list[str], operator.add]


def continue_to_jokes(state: OverallState):
    return [Send("generate_joke", {"subject": s}) for s in state['subjects']]


builder = StateGraph(OverallState)
builder.add_node("generate_joke", lambda state: {"jokes": [f"Joke about {state['subject']}"]})
builder.add_conditional_edges(START, continue_to_jokes)
builder.add_edge("generate_joke", END)
graph = builder.compile()

# Invoking with two subjects results in a generated joke for each
graph.invoke({"subjects": ["cats", "dogs", "pinguins", "birds", "dead", "alive"]})


{'subjects': ['cats', 'dogs', 'pinguins', 'birds', 'dead', 'alive'],
 'jokes': ['Joke about cats',
  'Joke about dogs',
  'Joke about pinguins',
  'Joke about birds',
  'Joke about dead',
  'Joke about alive']}

In [5]:
print(graph.get_graph().draw_ascii())

               +-----------+        
               | __start__ |        
               +-----------+        
              ...          ...      
             .                .     
           ..                  ...  
+---------------+                 . 
| generate_joke |              ...  
+---------------+             .     
              ***          ...      
                 *        .         
                  **    ..          
                +---------+         
                | __end__ |         
                +---------+         


In [ ]:
def my_node(state: State) -> Command[Literal["my_other_node"]]:
    return Command(
        # state update
        update={"foo": "bar"},
        # control flow
        goto="my_other_node"
    )

In [2]:
import requests
from bs4 import BeautifulSoup

In [5]:

# URL of the webpage you want to download
url = "https://www.youtube.com/playlist?list=PLfaIDFEXuae16n2TWUkKq5PgJ0w6Pkwtg"

# Step 1: Download the HTML page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    html_content = response.text
    
    # Step 2: Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Step 3: Extract all links
    links = [a['href'] for a in soup.find_all('a', href=True)]
    
    # Display the links
    for link in links:
        print(link)
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

/
/
https://www.youtube.com/about/
https://www.youtube.com/about/press/
https://www.youtube.com/about/copyright/
/t/contact_us/
https://www.youtube.com/creators/
https://www.youtube.com/ads/
https://developers.google.com/youtube
/t/terms
/t/privacy
https://www.youtube.com/about/policies/
https://www.youtube.com/howyoutubeworks?utm_campaign=ytgen&utm_source=ythp&utm_medium=LeftNav&utm_content=txt&u=https%3A%2F%2Fwww.youtube.com%2Fhowyoutubeworks%3Futm_source%3Dythp%26utm_medium%3DLeftNav%26utm_campaign%3Dytgen
/new


In [ ]:
from langchain import hub